In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)


In [ ]:

#!pip install --upgrade nltk
!pip install emoji

In [4]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from transformers import AutoConfig, AutoModel, AutoTokenizer, AdamW, get_scheduler
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import emoji

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
#root = '/content/drive/MyDrive/NLP/Davide/irony_davide_datasets/'
root = '/kaggle/input/irony-datasets/'

x_train = pd.read_csv(root + 'x_train.csv')
x_test = pd.read_csv(root + 'x_test.csv')
x_val = pd.read_csv(root + 'x_val.csv')
y_train = pd.read_csv(root + 'y_train.csv')
y_test = pd.read_csv(root + 'y_test.csv')
y_val = pd.read_csv(root + 'y_val.csv')

In [6]:
print(len(x_train))
print(len(x_test))
print(len(x_val))

2862
764
191


In [7]:
hyperparameters = {
    "epochs": 10,
    "learning_rate": 3e-5,   #1e-5
    "batch_size": 32,  #accura...
    "dropout": 0.1,   # o 0.3
    "weight_decay": 1e-4,  #1e-3
    "stopwords": False,
    "language_model": "microsoft/deberta-v3-base", #"vinai/bertweet-base", #bert-base-uncased
    "layers": 1,
    "h_dim": 768,
    "bilstm": True,
    "patience": 5,
    "min_delta": 0.01,
    "extra_features": 5,
    "threshold": 0.5
}

In [8]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, x, hashtag_count, avg_ironic_ratio, ironic_hashtag_count, non_ironic_hashtag_count, hashtag_irony_index, y, stopwords):  #nn_count
        # x e y sono series di pandas
        tokens_litt = [nltk.word_tokenize(text, language='english') for text in list(x)]
        text_clean = []

        if stopwords:
            for sentence in tqdm(tokens_litt, desc='Tokenizing ... '):
                text_clean.append(' '.join([w.lower() for w in sentence if
                                            not w.lower() in nltk.corpus.stopwords.words("english")]))
        else:
            for sentence in tqdm(tokens_litt, desc='Tokenizing ... '):
                text_clean.append(' '.join([w.lower() for w in sentence]))
            # ogni token è separato dall'altro con uno spazio

        self.texts = text_clean
        self.labels = [torch.tensor(label) for label in y]
        
        self.hashtag_count = [torch.tensor(count) for count in hashtag_count]
        self.avg_ironic_ratio = [torch.tensor(ratio) for ratio in avg_ironic_ratio]
        self.ironic_hashtag_count = [torch.tensor(count) for count in ironic_hashtag_count]
        self.non_ironic_hashtag_count = [torch.tensor(count) for count in non_ironic_hashtag_count]
        self.hashtag_irony_index = [torch.tensor(index) for index in hashtag_irony_index]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        batch_texts = self.texts[idx]
        batch_labels = np.array(self.labels[idx])
        
        batch_hashtag_count = np.array(self.hashtag_count[idx])
        batch_avg_ironic_ratio = np.array(self.avg_ironic_ratio[idx])
        batch_ironic_hashtag_count = np.array(self.ironic_hashtag_count[idx])
        batch_non_ironic_hashtag_count = np.array(self.non_ironic_hashtag_count[idx])
        batch_hashtag_irony_index = np.array(self.hashtag_irony_index[idx])

        return batch_texts, batch_hashtag_count, batch_avg_ironic_ratio, batch_ironic_hashtag_count, batch_non_ironic_hashtag_count, batch_hashtag_irony_index, batch_labels

In [9]:
#x_test

In [10]:
train_dataset = Dataset(x_train['text'], x_train['hashtag_count'], x_train['avg_ironic_ratio'], x_train['ironic_hashtag_count'], x_train['non_ironic_hashtag_count'], x_train['hashtag_irony_index'], y_train.squeeze(1), hyperparameters["stopwords"])
val_dataset = Dataset(x_val['text'], x_val['hashtag_count'], x_val['avg_ironic_ratio'], x_val['ironic_hashtag_count'], x_val['non_ironic_hashtag_count'], x_val['hashtag_irony_index'], y_val.squeeze(1), hyperparameters["stopwords"])
test_dataset = Dataset(x_test['text'], x_test['hashtag_count'], x_test['avg_ironic_ratio'], x_test['ironic_hashtag_count'], x_test['non_ironic_hashtag_count'], x_test['hashtag_irony_index'], y_test.squeeze(1), hyperparameters["stopwords"])

Tokenizing ... : 100%|██████████| 764/764 [00:00<00:00, 371616.40it/s]


In [11]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):

        self.patience = patience
        self.min_delta = min_delta              # valore minimo di decrescita della loss di validazione all'epoca corrente
                                                # per asserire che c'è un miglioramenti della loss
        self.counter = 0                        # contatore delle epoche di pazienza
        self.early_stop = False                 # flag di early stop
        self.min_validation_loss = torch.inf    # valore corrente ottimo della loss di validazione

    def __call__(self, validation_loss):
        # chiamata in forma funzionale dell'oggetto di classe EarlySopping

        if (validation_loss + self.min_delta) >= self.min_validation_loss:  # la loss di validazione non decresce
            self.counter += 1                                               # incrementiamo il contatore delle epoche di pazienza
            if self.counter >= self.patience:
                self.early_stop = True
                print("Early stop!")
        else:                                               # c'è un miglioramento della loss:
            self.min_validation_loss = validation_loss      # consideriamo la loss corrente
                                                            # come nuova loss ottimale
            self.counter = 0                                # e azzeriamo il contatore di pazienza


In [20]:
class ClassifierDeep(nn.Module):

    def __init__(self, hdim, dropout, model_name, extra_features = hyperparameters["extra_features"]): #hyperparameters["extra_features"]
        super(ClassifierDeep, self).__init__()
        config = AutoConfig.from_pretrained(model_name)
        self.lm_model = AutoModel.from_pretrained(model_name, config=config)
        self.classifier = nn.Sequential(
            nn.Linear(hdim + extra_features, 1),
            #nn.Dropout(dropout),
            #nn.ReLU(),
            #nn.Linear(16, 1),
            nn.Sigmoid()
            )


    def forward(self, input_id_text, attention_mask, hashtag_count, avg_ironic_ratio, ironic_hashtag_count, non_ironic_hashtag_count, hashtag_irony_index):
        output = self.lm_model(input_id_text, attention_mask).last_hidden_state
        output = output[:,0,:]
        #output = torch.cat((output, emoji_count.unsqueeze(1), has_ironic_emoji.unsqueeze(1), has_non_ironic_emoji.unsqueeze(1), nn_count.unsqueeze(1)), dim=1)
        output = torch.cat((output, hashtag_count.unsqueeze(1), avg_ironic_ratio.unsqueeze(1), ironic_hashtag_count.unsqueeze(1), non_ironic_hashtag_count.unsqueeze(1), hashtag_irony_index.unsqueeze(1)), dim=1)
        return self.classifier(output)

In [13]:
def train_loop(model, dataloader, tokenizer, loss, optimizer, device, threshold):
    model.train()

    epoch_acc = 0
    epoch_loss = 0

    for batch_texts, batch_hashtag_count, batch_avg_ironic_ratio, batch_ironic_hashtag_count, batch_non_ironic_hashtag_count, batch_hashtag_irony_index, batch_labels in dataloader:

        optimizer.zero_grad()

        tokens = tokenizer(list(batch_texts),
                           add_special_tokens=True,
                           return_tensors='pt',
                           padding='max_length',
                           max_length = 128,
                           truncation=True)

        input_id_texts = tokens['input_ids'].squeeze(1).to(device)
        mask_texts = tokens['attention_mask'].squeeze(1).to(device)
    
        batch_hashtag_count = batch_hashtag_count.float().to(device)
        batch_avg_ironic_ratio = batch_avg_ironic_ratio.float().to(device)
        batch_ironic_hashtag_count = batch_ironic_hashtag_count.float().to(device)
        batch_non_ironic_hashtag_count = batch_non_ironic_hashtag_count.float().to(device)
        batch_hashtag_irony_index = batch_hashtag_irony_index.float().to(device)
        batch_labels = batch_labels.float().to(device)


        output = model(input_id_texts, mask_texts, batch_hashtag_count, batch_avg_ironic_ratio, batch_ironic_hashtag_count, batch_non_ironic_hashtag_count, batch_hashtag_irony_index).squeeze(1)

        # la loss è una CrossEntropyLoss, al suo interno ha
        # la logsoftmax + negative log likelihood loss
        batch_loss = loss(output, batch_labels)
        batch_loss.backward()
        optimizer.step()

        epoch_loss += batch_loss.item()

        # per calcolare l'accuracy devo generare le predizioni
        # applicando manualmente la logsoftmax
        preds = (output > 0.6).float()  # Soglia di 0.5 per la classificazione binaria
        epoch_acc += (preds == batch_labels).sum().item()

        batch_labels = batch_labels.detach().cpu()
        input_id_texts = input_id_texts.detach().cpu()
        mask_texts = mask_texts.detach().cpu()
        
        batch_hashtag_count = batch_hashtag_count.detach().cpu()
        batch_avg_ironic_ratio = batch_avg_ironic_ratio.detach().cpu()
        batch_ironic_hashtag_count = batch_ironic_hashtag_count.detach().cpu()
        batch_non_ironic_hashtag_count = batch_non_ironic_hashtag_count.detach().cpu()
        batch_hashtag_irony_index = batch_hashtag_irony_index.detach().cpu()
        output = output.detach().cpu()


    return epoch_loss/len(dataloader), epoch_acc


In [14]:
from sklearn.metrics import f1_score

def test_loop(model, dataloader, tokenizer, loss, device, threshold):
    model.eval()

    epoch_acc = 0
    epoch_loss = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_texts, batch_hashtag_count, batch_avg_ironic_ratio, batch_ironic_hashtag_count, batch_non_ironic_hashtag_count, batch_hashtag_irony_index, batch_labels in dataloader:

            tokens = tokenizer(list(batch_texts),
                               add_special_tokens=True,
                               return_tensors='pt',
                               padding='max_length',
                               max_length=128,
                               truncation=True)
            input_id_texts = tokens['input_ids'].squeeze(1).to(device)
            mask_texts = tokens['attention_mask'].squeeze(1).to(device)
            batch_hashtag_count = batch_hashtag_count.float().to(device)
            batch_avg_ironic_ratio = batch_avg_ironic_ratio.float().to(device)
            batch_ironic_hashtag = batch_ironic_hashtag_count.float().to(device)
            batch_non_ironic_hashtag = batch_non_ironic_hashtag_count.float().to(device)
            batch_hashtag_irony_index = batch_hashtag_irony_index.float().to(device)
            batch_labels = batch_labels.float().to(device)

            output = model(input_id_texts, mask_texts, batch_hashtag_count, batch_avg_ironic_ratio, batch_ironic_hashtag, batch_non_ironic_hashtag, batch_hashtag_irony_index).squeeze(1)

            # Calcola la loss
            batch_loss = loss(output, batch_labels)
            epoch_loss += batch_loss.item()

            # Genera predizioni binarie
            preds = (output > 0.6).float()  #dovrei mettere threshold

            # Calcola l'accuracy
            epoch_acc += (preds == batch_labels).sum().item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())

            batch_labels = batch_labels.detach().cpu()
            input_id_texts = input_id_texts.detach().cpu()
            mask_texts = mask_texts.detach().cpu()
            batch_hashtag_count = batch_hashtag_count.detach().cpu()
            batch_avg_ironic_ratio = batch_avg_ironic_ratio.detach().cpu()
            batch_ironic_hashtag = batch_ironic_hashtag_count.detach().cpu()
            batch_non_ironic_hashtag = batch_non_ironic_hashtag_count.detach().cpu()
            batch_hashtag_irony_index = batch_hashtag_irony_index.detach().cpu()
            output = output.detach().cpu()

    f1 = f1_score(all_labels, all_preds)

    return epoch_loss/len(dataloader), epoch_acc, f1


In [15]:
'''def unfreeze_layers(model, freeze_up_to_layer):
    # Itera su tutti i parametri del modello
    for name, param in model.named_parameters():
        # Gestisce l'encoder e altri componenti
        if 'encoder.layer' in name:
            # Estrai il numero del layer
            try:
                layer_number = int(name.split('.')[2])  # esempio "encoder.layer.11.attention.self.query.weight"
            except ValueError:
                continue  # salta se non riesci a ottenere il numero del layer

            # Congela i parametri fino al livello specificato
            if layer_number < freeze_up_to_layer:
                param.requires_grad = False
            else:
                param.requires_grad = True

        # Gestione degli embeddings (se vuoi congelarli o meno)
        elif 'embeddings' in name:
            param.requires_grad = False  # Congela gli embeddings (o cambia se vuoi sbloccarli)

        # Gestione della parte del pooler e della testa di classificazione (se devi allenare queste parti)
        elif 'pooler' in name or 'classifier' in name:
            param.requires_grad = True  # Assicurati che questi componenti siano allenati

        # Stampa dello stato di "requires_grad" per ogni parametro
        print(f"{name} - requires_grad = {param.requires_grad}")
'''

'def unfreeze_layers(model, freeze_up_to_layer):\n    # Itera su tutti i parametri del modello\n    for name, param in model.named_parameters():\n        # Gestisce l\'encoder e altri componenti\n        if \'encoder.layer\' in name:\n            # Estrai il numero del layer\n            try:\n                layer_number = int(name.split(\'.\')[2])  # esempio "encoder.layer.11.attention.self.query.weight"\n            except ValueError:\n                continue  # salta se non riesci a ottenere il numero del layer\n\n            # Congela i parametri fino al livello specificato\n            if layer_number < freeze_up_to_layer:\n                param.requires_grad = False\n            else:\n                param.requires_grad = True\n\n        # Gestione degli embeddings (se vuoi congelarli o meno)\n        elif \'embeddings\' in name:\n            param.requires_grad = False  # Congela gli embeddings (o cambia se vuoi sbloccarli)\n\n        # Gestione della parte del pooler e della

In [16]:
def train_test(model, epochs, optimizer, device, threshold, train_data, test_data,
               batch_size, model_name, train_loss_fn,
               test_loss_fn=None,         # non necessariamente train e test loss devono differire
               early_stopping=None,       # posso addstrare senza early stopping
               val_data=None,             # e in questo caso non c'è validation set
               scheduler=None,            # possibile scheduler per monitorare l'andamento di un iperparametro
              ):    

    # Congelamento progressivo all'inizio
    #unfreeze_layers(model, freeze_up_to_layer)

    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

    # check sulle funzioni di loss
    #if test_loss_fn == None:
    #    test_loss_fn = train_loss_fn

    # liste dei valori di loss e accuracy epoca per epoca per il plot
    train_loss = []
    validation_loss = []
    test_loss = []
    train_acc = []
    validation_acc = []
    test_acc = []

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    all_emojis = list(emoji.EMOJI_DATA.keys())
    emoji_tokens = [emoji.demojize(e) for e in all_emojis]
    tokenizer.add_tokens(emoji_tokens)
    tokenizer.add_tokens(["@user"])
    model.lm_model.resize_token_embeddings(len(tokenizer))

    # Ciclo di addestramento con early stopping
    for epoch in range(1,epochs+1):

        epoch_train_loss, epoch_train_acc = train_loop(model, train_dataloader, tokenizer, train_loss_fn, optimizer, device, threshold)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_acc/len(train_data))

       # Validation se è presente la callback di early stopping
        if early_stopping != None:
            epoch_validate_loss, epoch_validate_acc, _, = test_loop(model, val_dataloader, tokenizer, test_loss_fn, device, threshold)
            validation_loss.append(epoch_validate_loss)
            validation_acc.append(epoch_validate_acc/len(val_data))

        # Test
        epoch_test_loss, epoch_test_acc, epcoch_f1 = test_loop(model, test_dataloader, tokenizer, test_loss_fn, device, threshold)
        test_loss.append(epoch_test_loss)
        test_acc.append(epoch_test_acc/len(test_data))


        val_loss_str = f'Validation loss: {epoch_validate_loss:6.4f} 'if early_stopping != None else ' '
        val_acc_str = f'Validation accuracy: {(epoch_validate_acc/len(val_data)):6.4f} ' if early_stopping != None else ' '
        print(f"\nTrain loss: {epoch_train_loss:6.4f} {val_loss_str} Test loss: {epoch_test_loss:6.4f}")
        print(f"Train accuracy: {(epoch_train_acc/len(train_data)):6.4f} {val_acc_str}Test accuracy: {(epoch_test_acc/len(test_data)):6.4f}")


        # Early stopping
        if early_stopping != None:
            early_stopping(epoch_validate_loss)
            if early_stopping.early_stop:
                break

        # Sblocca i layer ogni 'freeze_every_n_epochs'
        #if epoch % freeze_every_n_epochs == 0:
        #    freeze_up_to_layer = max(0, freeze_up_to_layer - 1)  # Sblocca un layer
        #    unfreeze_layers(model, freeze_up_to_layer)

    return train_loss, validation_loss, test_loss, train_acc, validation_acc, test_acc, epcoch_f1

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

model = ClassifierDeep(
                    hyperparameters["h_dim"],
                    hyperparameters["dropout"],
                    hyperparameters["language_model"]).to(device)
print(model)

# Calcoliamo il numero totale dei parametri del modello
total_params = sum(p.numel() for p in model.parameters())
print(f"Numbero totale dei parametri: {total_params}")

#threshold = hyperparameters['threshold']
criterion = nn.BCELoss()
optimizer = AdamW(model.parameters(), lr=hyperparameters['learning_rate'], weight_decay=hyperparameters['weight_decay'])

###### Linear Warmup + Decay ######
# Calcolo dei passi totali
#total_steps = len(train_dataset) // hyperparameters['batch_size'] * hyperparameters['epochs']

# Passi di warmup (ad esempio, 10% del totale)
#warmup_steps = int(0.1 * total_steps)

# Creazione del scheduler
#scheduler = get_scheduler(
#    name="linear",  # Tipo di scheduler   ---> PROVARE COSINE ---> provare con OTTIMIZZATORE SGD INVECE CHE ADAM
#    optimizer=optimizer,  # Ottimizzatore che stai usando
#    num_warmup_steps=warmup_steps,
#    num_training_steps=total_steps
#)
###################################


# Creiamo la callback di early stopping da passare al nostro metodo di addestramento
early_stopping = EarlyStopping(patience=hyperparameters['patience'], min_delta=hyperparameters['min_delta'])

Using cuda device
ClassifierDeep(
  (lm_model): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
#!export CUDA_LAUNCH_BLOCKING=1
#torch.cuda.empty_cache()

In [21]:
# Congela i layer fino al numero specificato
#freeze_up_to_layer = 4
#unfreeze_layers(model, freeze_up_to_layer)

# Recupera i parametri che richiedono il calcolo del gradiente (i parametri non congelati)
#params_to_train = [param for param in model.parameters() if param.requires_grad]

# Verifica se ci sono parametri da allenare
#if len(params_to_train) == 0:
#  raise ValueError("Non ci sono parametri da allenare. Verifica che il congelamento e sblocco dei layer siano corretti.")

# Creazione dell'ottimizzatore con i parametri non congelati
criterion = nn.BCELoss()
#optimizer = AdamW(params_to_train, lr=hyperparameters['learning_rate'])
optimizer = AdamW(model.parameters(), lr=hyperparameters['learning_rate'], weight_decay = hyperparameters['weight_decay'])

# Ora puoi chiamare la routine di addestramento
train_loss, validation_loss, test_loss, train_acc, validation_acc, test_acc, f1 = train_test(
    model,
    hyperparameters['epochs'],
    optimizer,
    device,
    hyperparameters['threshold'],
    train_dataset,
    test_dataset,  # Assicurati che questo sia il dataset corretto
    hyperparameters['batch_size'],
    hyperparameters['language_model'],
    criterion,
    criterion,
    early_stopping=early_stopping,
    val_data=val_dataset  # Qui passa il dataset di validazione
)
print(f'F1 score: {f1}')  

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x1 and 16x1)

In [ ]:
'''from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt

param_grid = {
    "learning_rate": [1e-4,1e-5,1e-6],
    "batch_size": [32,128],
    "dropout": [0.1, 0.2],
    "weight_decay": [1e-2,1e-4],
    "epochs": [5],
    "threshold": [0.4,0.6]
}

best_val_acc_acc=0
best_val_loss_acc=1
best_val_f1_acc=0
best_params_acc = {}
best_val_acc_loss=0
best_val_loss_loss=1
best_val_f1_loss=0
best_params_loss = {}
best_val_acc_f1=0
best_val_loss_f1=1
best_val_f1_f1=0
best_params_f1 = {}

grid = ParameterGrid(param_grid)
for params in grid:
  print("----------------------------------------------------------------")
  print(params)
  # Acquisiamo il device su cui effettueremo il training
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Using {device} device")

  model = ClassifierDeep(
                      hyperparameters["h_dim"],
                      hyperparameters["dropout"],
                      hyperparameters["language_model"]).to(device)
  print(model)

  # Calcoliamo il numero totale dei parametri del modello
  total_params = sum(p.numel() for p in model.parameters())
  print(f"Numbero totale dei parametri: {total_params}")

  criterion = nn.BCELoss()
  optimizer = AdamW(model.parameters(), lr=params["learning_rate"], weight_decay=params["weight_decay"])

  # Creiamo la callback di early stopping da passare al nostro metodo di addestramento
  early_stopping = EarlyStopping(patience=hyperparameters['patience'], min_delta=hyperparameters['min_delta'])




  train_loss, val_loss, test_loss, train_acc, val_acc, test_acc, f1 = train_test(
    model,
    hyperparameters['epochs'],
    optimizer,
    device,
    hyperparameters['threshold'],
    train_dataset,
    test_dataset,  # Assicurati che questo sia il dataset corretto
    hyperparameters['batch_size'],
    hyperparameters['language_model'],
    criterion,
    criterion,
    early_stopping=early_stopping,
    val_data=val_dataset  # Qui passa il dataset di validazione
  )

  print(f1)

  if val_acc[-1] > best_val_acc_acc:
    best_val_acc_acc = val_acc[-1]
    best_val_loss_acc = val_loss[-1]
    best_val_f1_acc = f1
    best_params_acc = params

  if val_loss[-1] < best_val_loss_loss:
    best_val_acc_loss = val_acc[-1]
    best_val_loss_loss = val_loss[-1]
    best_val_f1_loss = f1
    best_params_loss = params

  if f1 > best_val_f1_f1:
    best_val_acc_f1 = val_acc[-1]
    best_val_loss_f1 = val_loss[-1]
    best_val_f1_f1 = f1
    best_params_f1 = params

  fig, axs = plt.subplots(1, 2, figsize=(20, 10))

  axs[0].plot(train_loss, label='training loss')
  axs[0].plot(val_loss, label='validation loss')
  axs[0].legend(loc='upper right')
  axs[0].set_ylim(0,1)

  axs[1].plot(train_acc, label='training accuracy')
  axs[1].plot(val_acc, label='validation accuracy')
  axs[1].legend(loc='lower right')
  axs[1].set_ylim(0,1)

  plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].plot(train_loss, label='training loss')
axs[0].plot(validation_loss, label='validation loss')
axs[0].plot(test_loss, label='test loss')
axs[0].legend(loc='upper right')
axs[0].set_ylim(0,1)

axs[1].plot(train_acc, label='training accuracy')
axs[1].plot(validation_acc, label='validation accuracy')
axs[1].plot(test_acc, label='test accuracy')
axs[1].legend(loc='lower right')
axs[1].set_ylim(0,1)
